<a href="https://colab.research.google.com/github/Julialunna/Fed-DP-PSO/blob/main/PSO_SGD_centralized_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

FLPSO-ADAM : desbalanceamento mais extremo:

1) Primeiro, distribui 500 amostras de cada dígito para cada cliente.

2) Depois, com as 35000 amostras de treino restantes, distribuí da seguinte forma:

    - TODOS os dígitos 0 e 1 dessas 35000 foram para o cliente 0.

    - TODOS os dígitos 2 e 3 dessas 35000 foram para o cliente 1 e assim por diante.

Dessa forma, todos os clientes tem acesso um pouco de cada amostra, mas tem uma quantidade muito maior de amostras de dois dígitos específicos

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Subset, TensorDataset
from torchvision import datasets, transforms
import torch.nn.functional as F
import copy
import random
import torchvision
import torchvision.models as models
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import math
import numpy as np
from collections import OrderedDict

In [3]:
class MLP(nn.Module):

    def __init__(self, device, input_size=28*28, hidden_size=256, num_classes=10):
        super(MLP, self).__init__()
        self.device = device
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, num_classes)
        self.to(device)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Achatar o tensor de entrada
        y = self.fc1(x)
        y = self.relu(y)
        y = self.fc2(y)
        y = self.relu(y)
        y = self.fc3(y)

        return y

In [4]:
# Definições dos hiperparâmetros
NUM_CLIENTES = 5
NUM_PARTICULAS = 25
NUM_RODADAS = 50
NUM_DIGITOS = 10
INERCIA, C1, C2 = 0.8, 1.5,  1.9
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'training on {DEVICE}')


training on cuda


In [11]:
#Seeds para reprodutibilidade
random.seed(98)
torch.manual_seed(98)
torch.cuda.manual_seed(98)

class Particula:
    def __init__(self, particle_id, modelo_cliente):
        self.particle_id = particle_id
        self.device = modelo_cliente.device
        self.pesos = copy.deepcopy(modelo_cliente.state_dict())

        # Adiciona ruído leve nos pesos para quebrar simetria inicial
        for name in self.pesos:
            self.pesos[name] += 0.01 * torch.randn_like(self.pesos[name])

        self.melhor_pesos = copy.deepcopy(self.pesos)
        self.melhor_erro = float('inf')
        self.velocidade = {name: torch.zeros_like(param) for name, param in self.pesos.items()}

    def atualizar_pso(self, global_best_pesos, INERCIA, C1, C2):
        MAX_VELOCITY = 0.1  # Limite para evitar oscilações grandes
        for name in self.pesos:
            local_rand = random.random()
            global_rand = random.random()
            self.velocidade[name] = (
                INERCIA * self.velocidade[name]
                + C1 * local_rand * (self.melhor_pesos[name] - self.pesos[name])
                + C2 * global_rand * (global_best_pesos[name] - self.pesos[name])
            )

            # Clipping da velocidade
            self.velocidade[name] = torch.clamp(self.velocidade[name], -MAX_VELOCITY, MAX_VELOCITY)

            # Atualiza os pesos com a nova velocidade
            self.pesos[name] += self.velocidade[name]

    def avaliar_perda(self, modelo_cliente, criterio, dados):
        modelo_cliente.load_state_dict(self.pesos)
        modelo_cliente.eval()
        total_loss = 0

        with torch.no_grad():
            for inputs, labels in dados:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = modelo_cliente(inputs)
                loss = criterio(outputs, labels)
                total_loss += loss.item()

        return total_loss / len(dados)


def refinar_com_adam(self, criterio, train_loader, modelo, device, PSO):
        """Refina os pesos da melhor partícula usando Adam."""
        otimizador = optim.Adam(modelo.parameters(), lr=0.007, weight_decay=1e-5)
        modelo.train()
        for i in range(1):
          for inputs, labels in train_loader:
              inputs, labels = inputs.to(device), labels.to(device)
              otimizador.zero_grad()
              outputs = modelo(inputs)
              loss = criterio(outputs, labels)
              loss.backward()
              otimizador.step()

        # Atualiza os pesos da melhor partícula com os pesos refinados pelo Adam
        PSO.melhor_particula.melhor_pesos = copy.deepcopy(modelo.state_dict())
        PSO.melhor_particula.melhor_erro = calcular_loss(modelo, criterio, train_loader)

def calcular_loss( modelo, criterio, train_loader):
    modelo.eval()
    total_loss = 0

    with torch.no_grad():
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = modelo(inputs)
            loss = criterio(outputs, labels)
            total_loss += loss.item()

    return total_loss / len(train_loader)

class PSO:
    def __init__(self, modelo_global, dados, test, num_particulas):
        self.modelo = copy.deepcopy(modelo_global)  # Cada cliente tem seu próprio modelo
        self.num_particulas = num_particulas
        self.particulas = []
        #no set com flower no aprendizado federado atualizaremos melhor partícula com a melhor partícula de todos os clientes
        self.melhor_particula = None
        self.inicializar_particulas(num_particulas)
        self.dados = dados
        self.test = test

    def inicializar_particulas(self, num_particulas):
        """Cria um conjunto de partículas associadas ao cliente."""
        self.particulas = [Particula(i, self.modelo) for i in range(num_particulas)]

    def treinar_com_pso(self, INERCIA, C1, C2, criterio):
        """Treina as partículas usando PSO e atualiza a melhor partícula local."""
        global_best_pesos = self.melhor_particula.pesos if self.melhor_particula else self.particulas[0].pesos

        for particula in self.particulas:
            particula.atualizar_pso(global_best_pesos, INERCIA, C1, C2)
            erro = particula.avaliar_perda(self.modelo, criterio, self.dados)
            if erro < particula.melhor_erro:
                particula.melhor_erro = erro
                particula.melhor_pesos = copy.deepcopy(particula.pesos)

        self.selecionar_melhor_particula()


    def selecionar_melhor_particula(self):
        """Seleciona a melhor partícula do cliente."""
        self.melhor_particula = min(self.particulas, key=lambda p: p.melhor_erro)

def train_centralized(trainloader, testloader, epochs: int):

  # Criando o modelo global
  modelo = MLP(DEVICE)
  PSO_train = PSO(modelo, trainloader, testloader, NUM_PARTICULAS)
  criterio = nn.CrossEntropyLoss()
  for i in range(epochs):
    PSO_train.treinar_com_pso(INERCIA, C1, C2, criterio)
    modelo.load_state_dict(PSO_train.melhor_particula.pesos)
    refinar_com_adam(PSO_train.melhor_particula, criterio, trainloader, modelo, DEVICE, PSO_train)
    test_loss, test_accuracy = avaliar_modelo(modelo, criterio, testloader)
    print(f"Epoch {i+1}/{epochs} - Test Loss: {test_loss:.4f} - Test Accuracy: {test_accuracy:.4f}")

def avaliar_modelo(modelo, criterio, testloader):
    """Avalia o modelo global no conjunto de teste."""
    modelo.eval()  # Modo de avaliação
    total_loss = 0
    correct = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = modelo(inputs)
            loss = criterio(outputs, labels)

            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
            total_samples += labels.size(0)

    test_loss = total_loss / len(testloader)
    test_accuracy = (correct / total_samples)

    return test_loss, test_accuracy

mnist_test = torchvision.datasets.MNIST(root='./data', train=False, download=True)
X_test = mnist_test.data.view(-1, 28*28).numpy()  # Flatten
y_test = mnist_test.targets.numpy()

mnist_train = torchvision.datasets.MNIST(root='./data', train=True, download=True)
X_train = mnist_train.data.view(-1, 28*28).numpy()  # Flatten
y_train = mnist_train.targets.numpy()

# Normalização (como foi feito com o Iris)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Converter para tensores
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Criar datasets
trainset = TensorDataset(X_train, y_train)
testset = TensorDataset(X_test, y_test)
train_loader = DataLoader(trainset, batch_size=240, shuffle=True)
test_loader = DataLoader(testset, batch_size=240, shuffle=False)



# Executando o treinamento federado
train_centralized(train_loader, test_loader, 5)

Epoch 1/5 - Test Loss: 0.1767 - Test Accuracy: 0.9565
Epoch 2/5 - Test Loss: 0.1636 - Test Accuracy: 0.9560
Epoch 3/5 - Test Loss: 0.1737 - Test Accuracy: 0.9590
Epoch 4/5 - Test Loss: 0.1582 - Test Accuracy: 0.9622
Epoch 5/5 - Test Loss: 0.1688 - Test Accuracy: 0.9622


In [12]:
!pip install -q "flwr[simulation]" flwr-datasets
!pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.0/87.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.1/598.1 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is th

In [13]:
from flwr_datasets import FederatedDataset
from flwr_datasets.partitioner import IidPartitioner

NUM_PARTITIONS = 100

partitioner = IidPartitioner(num_partitions=NUM_PARTITIONS)
# Let's partition the "train" split of the MNIST dataset
# The MNIST dataset will be downloaded if it hasn't been already
fds = FederatedDataset(dataset="ylecun/mnist", partitioners={"train": partitioner})

In [14]:
# We could load a single partition like this
partition_0 = fds.load_partition(0)
partition_0

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/15.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['image', 'label'],
    num_rows: 600
})